In [506]:
import tweepy
from tweepy import API 
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from langdetect import detect

import numpy as np
import pandas as pd
 
# import twitter_credentials
consumer_key = "cFf7im7BH68xO9qh3zEsv3nFz"
consumer_secret = "2QyodEVK63XYE5D9RFPAo0I53rhBOsNocQGpGB8rapmEqxDnJi"
access_token = "1265319352795975680-EJBIU55ZHZnjn8svR420cqVuU9evRL"
access_token_secret = "i8h2utF6b2l2Uh7Vpg6c2mnPOO1CuthGdbyWtHSWdIURP"



# # # # TWITTER CLIENT # # # #
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)

        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets
    
# # # # TWITTER AUTHENTICATER # # # #
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        return auth

# # # # CREATING THE DATA FRAME # # # # 
class TweetAnalyzer():
    
    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])
        df['Date Created'] = np.array([tweet.created_at for tweet in tweets])
        df["24H"] = df["Date Created"].apply(lambda x: 0 if (datetime.utcnow()-x)>timedelta(1) else 1)
        df.drop(columns=["Date Created"],axis=1,inplace=True)
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df.drop(df[df['likes']<=1000].index,inplace=True)
        df.drop(columns="likes",inplace=True)

        return df
    
# # # # FEATURE EXTRACTION # # # #
class FilterData():
    '''
    Feature Extracting including removing Retweets, tweets which are in languages other than English and filtering out 
    tweets based on the text_query
    '''
    t = []
    
    def filter_RT():
        df["Retweets"] = df["Tweets"].apply(lambda x: True if x[:2]=="RT" else False)
        df.drop(df[df["Retweets"]==True].index,inplace=True)
        df.drop(columns=["Retweets"],axis=1,inplace=True)
        df.index = [i for i in range(df["Tweets"].count())]
    
    def filter_lang():
        df["Lang"] = df["Tweets"].apply(lambda x: False if detect(x)=="en" else True)
        df.drop(df[df["Lang"]==True].index,inplace=True)
        df.drop(columns=["Lang"],axis=1,inplace=True)
        df.index = [i for i in range(df["Tweets"].count())]
    
    def filter_tweets():
        text_query = ["#COVID-19","#COVID19","COVID-19","CORONA","CORONAINDIA","#COVID19INDIA","COVID19INDIA"]
        for i in df["Tweets"]:
            for j in text_query:
                if j in i:
                    t.append(i)
    
    def filter_c_tweets():
        df["C-Tweets"] = df["Tweets"].apply(lambda x: False if x in list(set(t)) else True)
        df.drop(df[df["C-Tweets"] == True].index,inplace=True)
        df.drop(columns="C-Tweets",axis=1,inplace=True)
        df.index = [i for i in range(df["Tweets"].count())]
        
    def filter_url():   
        urls = []
        df["Tweets"].apply(lambda x: urls.append(x[x.find("https://"):]) if "https://" in x else urls.append("False"))
        df["URL"] = urls
        
# # # # MAIN # # # #
if __name__ == '__main__':

    twitter_client = TwitterClient()
    
    tweet_analyzer = TweetAnalyzer()

    api = twitter_client.get_twitter_client_api()

    tweets = api.user_timeline(screen_name="narendramodi", count=200)

    df = tweet_analyzer.tweets_to_data_frame(tweets)
    FilterData.filter_RT()
    FilterData.filter_lang()
    FilterData.filter_tweets()
    FilterData.filter_c_tweets()
    FilterData.filter_url()
    df.to_csv('modi-tweets.csv')
df

,Tweets,24H,URL
0,Had a useful exchange with President Rodrigo D...,1,https://t.co/sAkeSHDCKE
1,"Thank you, my friend President @PaulKagame. \n...",0,https://t.co/9jrdBpmLcC
2,"Had an excellent talk with H.E. Filipe Nyusi, ...",0,https://t.co/i3LdEb1o80
3,"In this difficult time of COVID-19 pandemic, w...",0,https://t.co/Q4kGzq7yXX
4,"Thank you, Prime Minister @PKJugnauth for our ...",0,False
5,Had an excellent talk with President @Gotabaya...,0,https://t.co/teEjcKD3LI
6,Spoke to PM @Statsmin about the COVID-19 pande...,0,False
7,The post COVID-19 era will bring with it sever...,0,https://t.co/gwlAxduyJ1
8,Had an extensive meeting with CMs. Each CM sha...,0,https://t.co/53g7cuCHrP
9,"Today, technology is helping many in the effor...",0,https://t.co/DYNeGjce0p


# EXTRACT THE TWEET


In [351]:
# from selenium import webdriver
# import time
# link = 'https://t.co/WDn16d4Biz'
# driver = webdriver.Chrome(executable_path='C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe') #the path where chromedriver.exe is installed in your local system
# driver.get(link)
# time.sleep(10)
# elements = []
# for i in range(2, 4):
#     try:
#         elements.append(driver.find_elements_by_xpath('/html/body/div/div/div/div[2]/main/div/div/div/div/div/div/div/section/div/div/div/div[' + str(i) + ']/div/div/div/div/article/div/div[2]/div[2]/div[2]/div[1]/div/span'))
#     except:
#         continue
# for i in elements:
#     for j in i:
#         print(j.text)
# driver.close()

# CM OF KARNATAKA

In [352]:
# from selenium import webdriver
# import time
# link = 'https://twitter.com/CMofKarnataka/status/1270353657301741571'
# driver = webdriver.Chrome(executable_path='C:\MozScraper\Chrome\chromedriver.exe') #the path where chromedriver.exe is installed in your local system
# driver.get(link)
# time.sleep(10)
# elements = []

# try:
#     elements.extend(driver.find_elements_by_css_selector('span.css-901oao.css-16my406.r-1qd0xha.r-ad9z0x.r-bcqeeo.r-qvutc0'))
# except:
#     pass
# for i in elements:
#     print(i.text)
# driver.close()

# IMAGES

In [365]:
# from selenium import webdriver
# import time
# link = 'https://twitter.com/CMofKarnataka/status/1270353657301741571'
# driver = webdriver.Chrome(executable_path='C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe') #the path where chromedriver.exe is installed in your local system
# driver.get(link)
# time.sleep(10)
# elements = []
# images = []
# try:
#     elements.extend(driver.find_elements_by_class_name('css-9pa8cd'))
# except:
#     pass
# for i in elements:
#     images.append(i.get_attribute("src"))
# #driver.get(images[3])
# #images[3] contains the relevant image
# driver.close()
# images[3]

In [507]:
from selenium import webdriver
import time
def replace_tweet():
        url = df["URL"]
    for i in range(url.count()):
        driver = webdriver.Chrome(executable_path='C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe') #the path where chromedriver.exe is installed in your local system
        driver.get(url[i])
        time.sleep(10)
        elements = []
        for i in range(2, 4):
            try:
                elements.append(driver.find_elements_by_xpath('/html/body/div/div/div/div[2]/main/div/div/div/div/div/div/div/section/div/div/div/div[' + str(i) + ']/div/div/div/div/article/div/div[2]/div[2]/div[2]/div[1]/div/span'))
            except:
                continue
        for i in elements:
            for j in i:
                df.replace(to_replace = df["Tweets"][i],value = j.text,inplace=True)
#                 print(type(j.text))
        driver.close()
replace_tweet()
df.drop(columns=df[""])

In [510]:
df

,Tweets,24H,URL
0,Had a useful exchange with President Rodrigo D...,1,https://t.co/sAkeSHDCKE
1,"Thank you, my friend President @PaulKagame. \n...",0,https://t.co/9jrdBpmLcC
2,"Had an excellent talk with H.E. Filipe Nyusi, ...",0,https://t.co/i3LdEb1o80
3,"In this difficult time of COVID-19 pandemic, w...",0,https://t.co/Q4kGzq7yXX
4,"Thank you, Prime Minister @PKJugnauth for our ...",0,False
5,Had an excellent talk with President @Gotabaya...,0,https://t.co/teEjcKD3LI
6,Spoke to PM @Statsmin about the COVID-19 pande...,0,False
7,The post COVID-19 era will bring with it sever...,0,https://t.co/gwlAxduyJ1
8,Had an extensive meeting with CMs. Each CM sha...,0,https://t.co/53g7cuCHrP
9,"Today, technology is helping many in the effor...",0,https://t.co/DYNeGjce0p


In [512]:
df["Tweets"][0]

'Had a useful exchange with President Rodrigo Duterte about COVID-19 and other issues.  I thanked him for taking care of the Indian community in the Philippines.'

In [ ]:
import tweepy
from tweepy import API 
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from langdetect import detect

import numpy as np
import pandas as pd
 
# import twitter_credentials
consumer_key = "cFf7im7BH68xO9qh3zEsv3nFz"
consumer_secret = "2QyodEVK63XYE5D9RFPAo0I53rhBOsNocQGpGB8rapmEqxDnJi"
access_token = "1265319352795975680-EJBIU55ZHZnjn8svR420cqVuU9evRL"
access_token_secret = "i8h2utF6b2l2Uh7Vpg6c2mnPOO1CuthGdbyWtHSWdIURP"



# # # # TWITTER CLIENT # # # #
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)

        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets
    
# # # # TWITTER AUTHENTICATER # # # #
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        return auth

# # # # CREATING THE DATA FRAME # # # # 
class TweetAnalyzer():
    
    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])
        df['Date Created'] = np.array([tweet.created_at for tweet in tweets])
        df["24H"] = df["Date Created"].apply(lambda x: 0 if (datetime.utcnow()-x)>timedelta(1) else 1)
        df.drop(columns=["Date Created"],axis=1,inplace=True)
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df.drop(df[df['likes']<=1000].index,inplace=True)
        df.drop(columns="likes",inplace=True)

        return df
    
# # # # FEATURE EXTRACTION # # # #
class FilterData():
    '''
    Feature Extracting including removing Retweets, tweets which are in languages other than English and filtering out 
    tweets based on the text_query
    '''
    t = []
    
    def filter_RT():
        df["Retweets"] = df["Tweets"].apply(lambda x: True if x[:2]=="RT" else False)
        df.drop(df[df["Retweets"]==True].index,inplace=True)
        df.drop(columns=["Retweets"],axis=1,inplace=True)
        df.index = [i for i in range(df["Tweets"].count())]
    
    def filter_lang():
        df["Lang"] = df["Tweets"].apply(lambda x: False if detect(x)=="en" else True)
        df.drop(df[df["Lang"]==True].index,inplace=True)
        df.drop(columns=["Lang"],axis=1,inplace=True)
        df.index = [i for i in range(df["Tweets"].count())]
    
    def filter_tweets():
        text_query = ["#COVID-19","#COVID19","COVID-19","CORONA","CORONAINDIA","#COVID19INDIA","COVID19INDIA"]
        for i in df["Tweets"]:
            for j in text_query:
                if j in i:
                    t.append(i)
    
    def filter_c_tweets():
        df["C-Tweets"] = df["Tweets"].apply(lambda x: False if x in list(set(t)) else True)
        df.drop(df[df["C-Tweets"] == True].index,inplace=True)
        df.drop(columns="C-Tweets",axis=1,inplace=True)
        df.index = [i for i in range(df["Tweets"].count())]
        
    def filter_url():   
        urls = []
        df["Tweets"].apply(lambda x: urls.append(x[x.find("https://"):]) if "https://" in x else urls.append("False"))
        df["URL"] = urls
        
# # # # MAIN # # # #
if __name__ == '__main__':

    twitter_client = TwitterClient()
    
    tweet_analyzer = TweetAnalyzer()

    api = twitter_client.get_twitter_client_api()

    tweets = api.user_timeline(screen_name="narendramodi", count=200)

    df = tweet_analyzer.tweets_to_data_frame(tweets)
    FilterData.filter_RT()
    FilterData.filter_lang()
    FilterData.filter_tweets()
    FilterData.filter_c_tweets()
    FilterData.filter_url()
    df.to_csv('modi-tweets.csv')
df